# Исследование надежности заемщиков.

*Цель исследования* - определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

*Заказчик* - кредитный отдел банка.

*Практическое применение* - результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Общие выводы:  клиенты банка достаточно дисциплинированы - 90% заемщиков не допускает просрочек по кредитам. Самые дисциплинированные категории: заемщики без детей, вдовцы и вдовы, с уровнем дохода в диапазоне от 30001–50000, берущие кредит на операции с недвижимостью.**

### Шаг 1. Обзор данных

Данные предоставлены в файле data.csv.
Описание данных:
- children — количество детей в семье;
- days_employed — общий трудовой стаж в днях;
- dob_years — возраст клиента в годах;
- education — уровень образования клиента;
- education_id — идентификатор уровня образования;
- family_status — семейное положение;
- family_status_id — идентификатор семейного положения;
- gender — пол клиента;
- income_type — тип занятости;
- debt — имел ли задолженность по возврату кредитов;
- total_income — ежемесячный доход;
- purpose — цель получения кредита/

In [1]:
# импорт библиотек
import pandas as pd
from IPython.display import display

In [2]:
# загрузка данных
data = pd.read_csv('/datasets/data.csv')

In [3]:
# просмотр первых строк и шапки данных
data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


In [4]:
# просмотр статистических характеристик
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
# просмотр информации о данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выводы:
1. Из 12 переменных только 2 содержат пропуски. Это days_employed и total_income. Число их одинаково, что говорит о том, что среди клиентов банка нет самозанятых.
2. Переменные children и days_employed содержат, как отрицательные значения, так и аномально высокие. Необходима корректировка данных.
3. Переменная dob_years содержит значение 0, что невозможно для клиента банка. Необходима корректировка данных.
4. Переменная days_employed как минимум 75% отрицательных значений. Очевидно, что произошла ошибка при выгрузке данных. Необходимо уточнить у специалиста, ответственного за формирование данных. 
5. Переменная education содержит не стандартизированные записи о уровне образования. Необходимо привести к единому регистру.

### Шаг 2.1 Заполнение пропусков

Пропуски есть только в переменных days_employed и total_income. При этом, days_employed содержит такой набор данных, что минимум 75% -  отрицательные значения. days_employed не важна для целей проекта, исключаем из анализа, но оставляем в наборе данных.
Чуть более 10% данных в столбце total_income отсутствуют. Заменим пропуски на медианное значение по подгруппам income_type

In [6]:
#замена пропусков в total_income на медианное значение по подгруппам income_type
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
data['total_income'].isna().sum()

0

### Шаг 2.2 Проверка данных на аномалии и исправления.

В столбце children максимальное значение - 20, минимальное - (-1). Вероятно, имеет место ошибка ввода данных. Необходимо заменить на 2 и 1.

In [7]:
# замена максимального и минимального значений столбца children
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

# проверка уникальный значений столбца children
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [8]:
# если в столбце наличие 5 детей встречается не единожды, оставляем значение 5
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Столбец days_employed пропускаем, как полностью неверный, но в исследовании не нужный. На данный момент нет возможности получить данные в другом виде, а для анализа нет необходимости корректировать имеющиеся. 
Переменная спорная - отражает общий трудовой стаж, что никак не иллюстрирует заинтересованность работодателя в заемщике на данный момент.

В столбце dob_years - возраст заемщика,  минимальное значение 0. Посмотрим частоту значений переменной.

In [9]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


101 значение равно 0, при этом, среднее арифметическое и медианное значения достаточно близки. Заменим 0 на медиану. Изменим тип переменной.

In [10]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].median()
data['dob_years'].sort_values().unique()

array([19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31.,
       32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44.,
       45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57.,
       58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69., 70.,
       71., 72., 73., 74., 75.])

In [11]:
# посмотрим уникальные значения education
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [12]:
# приведем к нижнему регистру
data['education'] = data['education'].str.lower()
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [13]:
# посмотрим уникальные значения family_status
data['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [14]:
# приведем к нижнему регистру
data['family_status'] = data['family_status'].str.lower()
data['family_status'].sort_values().unique()

array(['в разводе', 'вдовец / вдова', 'гражданский брак',
       'женат / замужем', 'не женат / не замужем'], dtype=object)

In [15]:
# посмотрим уникальные значения gender
data['gender'].sort_values().unique()

array(['F', 'M', 'XNA'], dtype=object)

В гендерном распределении есть значение XNA, вероятно это комбинация Now Answer и X. Посмотрим количество таких данных.

In [16]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [17]:
# XNA - 1 раз встречается, удаляем.
data = data[data['gender'] != 'XNA']
print(data['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64


In [18]:
# посмотрим уникальные значения income_type
data['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [19]:
#income_type не меняем, проверим распределение
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


### Шаг 2.3. Изменение типов данных.

Изменение типов данных необходимо для переменных: dob_years и total_income. В первом случае не нужна дробная часть, во втором - float лучше заменить на int

In [20]:
# изменение типа данных dob_years
data['dob_years'] = data['dob_years'].astype('int64')
data['dob_years'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75])

In [21]:
# изменение типа данных total_income
data['total_income'] = data['total_income'].astype('int64')

### Шаг 2.4. Удаление дубликатов.

In [22]:
# удаление дубликатов
data.duplicated().sum()
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21452 entries, 0 to 21524
Data columns (total 12 columns):
children            21452 non-null int64
days_employed       19350 non-null float64
dob_years           21452 non-null int64
education           21452 non-null object
education_id        21452 non-null int64
family_status       21452 non-null object
family_status_id    21452 non-null int64
gender              21452 non-null object
income_type         21452 non-null object
debt                21452 non-null int64
total_income        21452 non-null int64
purpose             21452 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Необходимо создать словари по уровню образования и семейному положению.

In [23]:
# словарь по уровню образования
edu_dict = data[['education_id', 'education']]
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True)
data.drop(columns = ['education'],axis = 1, inplace=True)

In [24]:
data.head()

,children,days_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [25]:
# словарь по семейному положению
fam_dict = data[['family_status_id', 'family_status']]
fam_dict = fam_dict.drop_duplicates().reset_index(drop=True)
data.drop(columns = ['family_status'],axis = 1, inplace=True)

In [26]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [27]:
# необходимо выделить следующие категории дохода:
# 0–30000 — 'E';
# 30001–50000 — 'D';
# 50001–200000 — 'C';
# 200001–1000000 — 'B';
# 1000001 и выше — 'A'
def income_level(income):
    if income < 30000:
        return 'E'
    if 30001 < income < 50000:
        return 'D'
    if 50001 < income < 200000:
        return 'C'
    if 200001 < income < 1000000:
        return 'B'
    return 'A'
data['income_level'] = data['total_income'].apply(income_level)
data.head()
        


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_level
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [28]:
from pymystem3 import Mystem
m = Mystem()
# находим уникальные значения в столбце 
lem_purpose = data['purpose'].unique()
print(lem_purpose)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Все цели кредита необходимо разбить на следующие группы:
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [29]:
def lemmas_purpose(purpose):
    lemma = m.lemmatize(purpose)
    for value in lemma:
        if 'автомоб' in value:
            return 'операции с автомобилем'    
        elif 'жил' in value or 'недвиж' in value:
            return 'операции с недвижимостью'
        elif 'свадь' in value:
            return 'проведение свадьбы'
        elif 'образов' in value:
            return 'получение образования'   
      

    
data['lemmas_purpose'] = data['purpose'].apply(lemmas_purpose)

data['lemmas_purpose'].value_counts()

операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: lemmas_purpose, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: наличие зависимости между количеством детей и возвратом кредита в срок

Есть ли зависимость между количеством детей и возвратом кредита в срок?
Для ответа на этот вопрос можно построить сводную таблицу и определить долю клиентов, имеющих просроченные платежи.

In [30]:
# сводная таблица наличия детей в семье заемщика и наличия просроченных платежей по кредиту
data_children_pivot = data.pivot_table(index=['children'], columns='debt', values='gender', aggfunc='count')
data_children_pivot = data_children_pivot.fillna(0)
display(data_children_pivot)

debt,0,1
children,,
0,13026.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,0.0


In [31]:
# доля клиентов, имеющих просроченные платежи и ее распределение в зависимости от числа детей в семье заемщика
debt_child = data.pivot_table(index=['children'], columns='debt', values='gender', aggfunc='count')
display(debt_child[1] / (debt_child[0]+debt_child[1]) *100)

children
0    7.544893
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5         NaN
dtype: float64

Вывод 1: Можно говорить о достаточно высокой дисциплине среди заемщиков банка - 90% и более заемщиков исправно платят по кредитам. Среди клиентов самая дисциплинированная группа заемщиков - группа без детей, на втором месте - с 3 детьми.  При определении кредитного рейтинга, группе, не имеющей детей можно присвоить ранг 1, группе с 3 детьми - ранг 2, с 1 ребенком - ранг 3, с 2 детьми - ранг 4. В велом же, разница между группами несущественная.

##### Вопрос 2:  наличие зависимости между семейным положением и возвратом кредита в срок

Есть ли зависимость между семейным положением и возвратом кредита в срок?
Для ответа на этот вопрос можно построить сводную таблицу и определить долю клиентов, имеющих просроченные платежи.

In [32]:
# сводная таблица семейного положения заемщика и наличия просроченных платежей по кредиту
data_family_pivot = data.pivot_table(index=['family_status_id'], columns='debt', values='gender', aggfunc='count')
data_family_pivot = data_family_pivot.fillna(0)
res_family = data_family_pivot.merge(fam_dict, on='family_status_id', how='left')
display(res_family)

,family_status_id,0,1,family_status
0,0,11408,931,женат / замужем
1,1,3761,388,гражданский брак
2,2,896,63,вдовец / вдова
3,3,1110,85,в разводе
4,4,2536,274,не женат / не замужем


In [33]:
# доля клиентов, имеющих просроченные платежи и ее распределение в зависимости от семейного положения заемщика
debt_family = data.pivot_table(index=['family_status_id'], columns='debt', values='gender', aggfunc='count')
display(debt_family[1] / (debt_family[0]+debt_family[1]) *100)

family_status_id
0    7.545182
1    9.351651
2    6.569343
3    7.112971
4    9.750890
dtype: float64

Вывод 2: Среди клиентов, самая дисциплинированная группа заемщиков, в зависимости от семейного положения - "вдовец/вдова", на втором месте - "в разводе". Разница между группами несущественная. Разница между частотой возникновения просрочек по кредитам между группами говорит от обсутствии какой-либо связи между возвратом кредита и семейным положением.

##### Вопрос 3: наличие зависимости между уровнем дохода и возвратом кредита в срок

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Для ответа на этот вопрос можно построить сводную таблицу и определить долю клиентов, имеющих просроченные платежи.

In [34]:
# сводная таблица уровня дохода заемщика и наличия просроченных платежей по кредиту
data_income_pivot = data.pivot_table(index=['income_level'], columns='debt', values='gender', aggfunc='count')
data_income_pivot = data_income_pivot.fillna(0)
display(data_income_pivot)

debt,0,1
income_level,,
A,24,2
B,4684,356
C,14654,1360
D,329,21
E,20,2


In [35]:
# доля клиентов, имеющих просроченные платежи и ее распределение в зависимости от уровня дохода
debt_income = data.pivot_table(index=['income_level'], columns='debt', values='gender', aggfunc='count')
display(debt_income[1] / (debt_income[0]+debt_income[1]) *100)

income_level
A    7.692308
B    7.063492
C    8.492569
D    6.000000
E    9.090909
dtype: float64

Вывод 3: Среди клиентов, самая дисциплинированная группа заемщиков, в зависимости от уровня дохода - "D (30001–50000)". В целом же разница между группами несущественная.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок

In [36]:
# доля клиентов, имеющих просроченные платежи и ее распределение в зависимости от семейного положения заемщика
debt_income = data.pivot_table(index=['income_level'], columns='debt', values='gender', aggfunc='count')
display(debt_income[1] / (debt_income[0]+debt_income[1]) *100)

income_level
A    7.692308
B    7.063492
C    8.492569
D    6.000000
E    9.090909
dtype: float64

Есть ли зависимость между целями кредита и возвратом кредита в срок?
Для ответа на этот вопрос можно построить сводную таблицу и определить долю клиентов, имеющих просроченные платежи.

In [37]:
# сводная таблица целей кредита и наличия просроченных платежей по кредиту
data_purpose_pivot = data.pivot_table(index=['lemmas_purpose'], columns='debt', values='gender', aggfunc='count')
data_purpose_pivot = data_purpose_pivot.fillna(0)
display(data_purpose_pivot)

debt,0,1
lemmas_purpose,,
операции с автомобилем,3903,403
операции с недвижимостью,10028,782
получение образования,3643,370
проведение свадьбы,2137,186


In [38]:
# доля клиентов, имеющих просроченные платежи и ее распределение в зависимости от целей кредита
debt_purpose = data.pivot_table(index=['lemmas_purpose'], columns='debt', values='gender', aggfunc='count')
display(debt_purpose[1] / (debt_purpose[0]+debt_purpose[1]) *100)

lemmas_purpose
операции с автомобилем      9.359034
операции с недвижимостью    7.234043
получение образования       9.220035
проведение свадьбы          8.006888
dtype: float64

Вывод 4: Чаще остальных, клиенты, оформляющие кредит на операции с недвижимостью не допускают просрочек.

### Общий вывод:

В целом, клиенты банка достаточно дисциплинированы - 90% заемщиков не допускает просрочек по кредитам. Самые дисциплинированные категории:
- заемщики без детей; 
- вдовцы и вдовы; 
- с уровнем дохода в диапазоне от 30001–50000;
- берущие кредит на операции с недвижимостью.

Наименее дисциплинированные:

- заемщики с 4 детьми;
- не женатые/замужние;
- с уровнем дохода 0–30000;
- берущие кредит на операции с автомобилем.

Явной зависимости возврата кредита в срок и описанными выше характеристиками заемщика нет.